In [1]:
import scanpy as sc

In [20]:
import decoupler as dc

In [42]:
import pandas as pd

In [3]:
adata = sc.read_h5ad("adata_solo_annotated.h5ad")

In [5]:
adata

AnnData object with n_obs × n_vars = 60459 × 11945
    obs: '_scvi_batch', '_scvi_labels', 'batch', 'batch_id', 'cell_type', 'condition', 'is_doublet', 'is_outlier', 'is_outlier_counts', 'is_outlier_genes', 'is_outlier_mito', 'is_outlier_top_20', 'leiden', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_ribo', 'n_counts', 'n_genes', 'n_genes_by_counts', 'origin', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'sample_id', 'total_counts', 'total_counts_mt', 'total_counts_ribo', 'value', 'outlier', 'mt_outlier', 'ribo_outlier'
    var: 'ensembl_id', 'feature_types', 'gene_name', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'cell_type_colors', 'hvg', 'is_doublet_colors', 'leiden', 'log1p', 'neighbo

In [23]:
pdata = dc.get_pseudobulk(
    adata,
    sample_col='sample_id',
    groups_col='cell_type',
    layer='counts',
    min_cells=0,
    min_counts=0
)

In [33]:
path = "/data/scratch/kvalem/projects/2021/honda_microbial_metabolites_2021/40_tables/40_single-cell-sorted-cd8/40_gex_surface_prot"

In [34]:
### Samplesheet
samplesheet = pdata.obs.copy()
samplesheet["sample_id"]=samplesheet.index
samplesheet.reset_index(inplace=True)

In [35]:
samplesheet.to_csv(f"{path}/samplesheet_ps.csv", index=False)
samplesheet.to_csv(f"{path}/samplesheet_ps.tsv", index=False, sep = "\t")

In [36]:
bulk_df = pdata.to_df().T

In [37]:
### Raw counts
bulk_df = pdata.to_df().T
#bulk_df["gene_id"]=bulk_df.index
bulk_df.columns = bulk_df.columns.str.replace("[.-]", "_").str.replace(" ", "_")

/tmp/ipykernel_1906231/3249437527.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  bulk_df.columns = bulk_df.columns.str.replace("[.-]", "_").str.replace(" ", "_")


In [38]:
bulk_df.to_csv(f"{path}/counts_ps.csv")

In [41]:
scales_counts = sc.pp.normalize_total(pdata, target_sum=None, inplace=False)
# log1p transform
pdata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [43]:
### log1p_norm counts
pdata_log1p_norm = pd.DataFrame(pdata.layers["log1p_norm"], 
    pdata.to_df().index, 
    pdata.to_df().columns)
bulk_df = pdata_log1p_norm.T
#bulk_df["gene_id"]=bulk_df.index
bulk_df.columns = bulk_df.columns.str.replace("[.-]", "_").str.replace(" ", "_")
bulk_df.to_csv(f"{path}/log1p_norm_counts_ps.csv")   

/tmp/ipykernel_1906231/358725614.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  bulk_df.columns = bulk_df.columns.str.replace("[.-]", "_").str.replace(" ", "_")
